In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib as mpl
import seaborn as sns
import plotly
from sklearn.model_selection import train_test_split

In [3]:
contraceptive_data = pd.read_csv("contraceptive/contraceptive_for_students.csv")
contraceptive_data

,wife_age,wife_education,husband_education,num_child,wife_religion,wife_work,husband_occupation,standard_living,media_exposure,contraceptive
0,24,2,3,3,1,1,2,3,0,1
1,45,1,3,10,1,1,3,4,0,1
2,43,2,3,7,1,1,3,4,0,1
3,42,3,2,9,1,1,3,3,0,1
4,36,3,3,8,1,1,3,2,0,1
...,...,...,...,...,...,...,...,...,...,...
1468,33,4,4,2,1,0,2,4,0,3
1469,33,4,4,3,1,1,1,4,0,3
1470,39,3,3,8,1,0,1,4,0,3
1471,33,3,3,4,1,0,2,2,0,3


In [4]:
train, test = train_test_split(contraceptive_data, test_size=0.2)

In [37]:
# Exploratory Data Analysis - Needs Work
numerical = ['wife_age', 'num_child']
categorical = ['wife_education', 'husband_education', 'wife_religion', 'wife_work', 'husband_occupation', 'standard_living', 'media_exposure']
ordinal = ['wife_education', 'husband_education', 'standard_living']
nominal = ['wife_religion', 'wife_work', 'husband_occupation', 'media_exposure']
all_vars = numerical + categorical

In [38]:
# Data Cleaning
from sklearn.feature_extraction import DictVectorizer
valid_entries = {'wife_education':[1, 2, 3, 4],'husband_education':[1, 2, 3, 4], 'wife_religion':[0,1], 'wife_work':[0,1], 'husband_occupation':[1, 2, 3, 4], 'standard_living':[1, 2, 3, 4], 'media_exposure':[0,1], 'contraceptive':[1,2,3]}
def clean_data(dataset):
    dataset = dataset.copy()
    # Remove rows violating constraints
    dataset = dataset.fillna(-1)
    dataset = dataset.astype(int)
    dataset = dataset[(dataset['wife_age']>0)&(dataset['num_child']>0)&(dataset['wife_education'].isin(valid_entries['wife_education']))&(dataset['husband_education'].isin(valid_entries['husband_education']))&(dataset['wife_religion'].isin(valid_entries['wife_religion']))&(dataset['wife_work'].isin(valid_entries['wife_work']))&(dataset['husband_occupation'].isin(valid_entries['husband_occupation']))&(dataset['standard_living'].isin(valid_entries['standard_living']))&(dataset['media_exposure'].isin(valid_entries['media_exposure']))&(dataset['contraceptive'].isin(valid_entries['contraceptive']))]
    # One-hot encode categorical variables
    dataset[categorical] = dataset[categorical].astype(str)
    x, y = dataset[all_vars], np.array(dataset['contraceptive'])
    records = x.to_dict(orient='records')
    encoder = DictVectorizer(sparse=False)
    x = encoder.fit_transform(records)
    return (x, y) 

In [51]:
# Model Generation - classification task -> Logistic Regression, Decision Trees, more?
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
x_train, y_train = clean_data(train)
logistic_model = LogisticRegressionCV(multi_class='ovr')
logistic_model.fit(x_train, y_train)
randomForest_model = RandomForestClassifier()
randomForest_model.fit(x_train, y_train)
(logistic_model.score(x_train, y_train), randomForest_model.score(x_train, y_train))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mode

(0.5108695652173914, 0.9565217391304348)

In [52]:
# Model Testing
x_test, y_test = clean_data(test)
(logistic_model.score(x_test, y_test), randomForest_model.score(x_test, y_test))

(0.5110294117647058, 0.4852941176470588)

In [ ]:
# Interpret Results